In [17]:
import pandas as pd
import numpy as np

In [18]:
dist_df = pd.read_csv('data/140_nodes/distance_matrix.csv').drop(columns=['distance', 'time', 'price_per_km'])
dist_df

,src,dst,price
0,12652,12652,0.000000e+00
1,12652,15008,8.685399e+06
2,12652,11112,9.685836e+06
3,12652,16325,5.617066e+06
4,12652,11561,1.556072e+08
...,...,...,...
19595,15976,17781,1.499276e+08
19596,15976,19971,1.837850e+08
19597,15976,16887,1.240729e+08
19598,15976,10985,2.743305e+08


In [19]:
office_df = pd.read_csv('data/140_nodes/offices.csv').drop(columns=['Unnamed: 0', 'office_name'])
office_df

,office_id,transfer_price,transfer_max
0,12652,100.0,300.0
1,15008,100.0,300.0
2,11112,100.0,300.0
3,16325,100.0,300.0
4,11561,100.0,300.0
...,...,...,...
135,17781,100.0,300.0
136,19971,100.0,300.0
137,16887,100.0,300.0
138,10985,100.0,300.0


In [20]:
reqs_df = pd.read_csv('data/140_nodes/reqs.csv')
reqs_df

,src_office_id,dst_office_id,volume
0,12652,16339,1
1,15008,17223,287
2,11112,13165,300
3,16325,16117,122
4,11561,16236,1
...,...,...,...
3984,11209,16962,184
3985,11147,18219,87
3986,18542,11200,100
3987,10963,14519,91


In [21]:
reqs_df['volume'].sum()

np.int64(523693)

In [22]:
import pandas as pd
import networkx as nx
import pulp as pl
from itertools import islice

In [23]:
DIST_CSV = "data/140_nodes/distance_matrix.csv"
OFFICE_CSV = "data/140_nodes/offices.csv"
REQS_CSV = "data/140_nodes/reqs.csv"

C = 90                # vehicle capacity
K_INIT_PATHS = 5      # number of initial paths per request
MAX_CG_ITERS = 200    # maximum number of CG iterations
EPS = 1e-8            # tolerance for reduced cost (seek < -EPS)
TIME_LIMIT_LP = 300   # seconds limit for LP stage (optional)
TIME_LIMIT_MIP = 600  # seconds limit for final MIP

In [24]:
dist_df = pd.read_csv(DIST_CSV)      # src,dst,price
office_df = pd.read_csv(OFFICE_CSV)  # office_id,transfer_price,transfer_max
reqs_df = pd.read_csv(REQS_CSV)      # src_office_id,dst_office,volume

# build directed graph
G = nx.DiGraph()
for _, row in dist_df.iterrows():
    u, v = row['src'], row['dst']
    G.add_edge(u, v, price=float(row['price']))

# offices data
transfer_price = office_df.set_index('office_id')['transfer_price'].to_dict()
transfer_max = office_df.set_index('office_id')['transfer_max'].to_dict()

# requests list
reqs = []
for idx, r in reqs_df.iterrows():
    reqs.append({
        'id': int(idx),
        's': r['src_office_id'],
        't': r['dst_office_id'],
        'd': float(r['volume'])
    })

In [25]:
def transit_cost_of_path(path, transfer_price_map):
    # path: list of nodes [v0, v1, ..., vn], exclude endpoints
    return sum(transfer_price_map.get(v, 0.0) for v in path[1:-1])

# helper: shortest simple paths generator limited to k
def k_shortest_simple_paths(G_in, source, target, k=1, weight='weight'):
    # networkx.shortest_simple_paths returns generator sorted by path weight
    try:
        gen = nx.shortest_simple_paths(G_in, source, target, weight=weight)
        return list(islice(gen, k))
    except Exception:
        return []

In [26]:
paths_for_req = {}  # dict: req_id -> list of paths (each path is list of nodes)
for r in reqs:
    s, t = r['s'], r['t']
    # build H where edge weight = price + transfer_price[dst] unless dst==t
    H = nx.DiGraph()
    for u, v, dat in G.edges(data=True):
        node_transit = transfer_price.get(v, 0.0) if v != t else 0.0
        w = dat['price'] + node_transit
        H.add_edge(u, v, weight=w, price=dat['price'])
    sel = k_shortest_simple_paths(H, s, t, k=K_INIT_PATHS, weight='weight')
    if H.has_edge(s, t):
        sel.append([s, t])
        print('added')
    if len(sel) == 0:
        raise ValueError(f"No path found for request {r['id']} from {s} to {t}")
    paths_for_req[r['id']] = sel

added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
adde

In [27]:
sel

[[np.int64(13665), np.float64(13165.0), np.float64(13170.0), np.int64(16236)],
 [np.int64(13665),
  np.float64(13165.0),
  np.float64(13170.0),
  np.float64(14060.0),
  np.int64(16236)],
 [np.int64(13665), np.float64(13165.0), np.float64(18812.0), np.int64(16236)],
 [np.int64(13665),
  np.float64(13165.0),
  np.float64(18812.0),
  np.float64(14060.0),
  np.int64(16236)],
 [np.int64(13665), np.float64(13165.0), np.float64(12746.0), np.int64(16236)],
 [np.int64(13665), np.int64(16236)]]

In [28]:
import pulp as pl

def build_initial_master(G, reqs, paths_for_req, C, transfer_price, transfer_max, TIME_LIMIT_LP=600):
    """
    Creates the initial master LP model for Column Generation using PuLP.
    
    Parameters:
    - G: NetworkX DiGraph with 'price' attribute for edges
    - reqs: list of dicts with keys 'id', 's' (source), 't' (target), 'd' (demand)
    - paths_for_req: dict {rid: [ [path1], [path2], ... ]}
    - C: capacity of one vehicle
    - transfer_price: dict {v: transfer cost at node v}
    - transfer_max: dict {v: max incoming capacity at node v}
    - TIME_LIMIT_LP: timeout for LP (seconds)
    
    Returns:
    - prob: PuLP model
    - x_vars: dict of flow variables {(rid, pid): {'var': var, 'path': path}}
    - y_vars: dict of vehicle variables {(u,v): var}
    - demand_constr, edgecap_constr, node_in_constr: dicts of constraints
    """

    prob = pl.LpProblem("master_lp", pl.LpMinimize)

    # Set solver with time limit
    solver = pl.PULP_CBC_CMD(msg=0, timeLimit=TIME_LIMIT_LP, warmStart=True)
    prob.setSolver(solver)

    # Create y variables for edges (vehicle usage)
    y_vars = {}
    for u, v in G.edges():
        y = pl.LpVariable(f"y_{u}_{v}", lowBound=0, cat='Continuous')
        y_vars[(u, v)] = y

    # Create x variables and collect expressions for constraints
    x_vars = {}
    demand_constr = {}     # rid -> LpConstraint
    edgecap_constr = {}    # (u,v) -> LpConstraint
    node_in_constr = {}    # v -> LpConstraint

    for r in reqs:
        rid = r['id']
        s_r = r['s']
        t_r = r['t']
        d_r = r['d']

        # Initialize demand expression
        demand_expr = pl.LpAffineExpression()

        for pid, path in enumerate(paths_for_req[rid]):
            # Transit cost: sum transfer_price for transit nodes (excluding source and target)
            transit_cost = sum(transfer_price.get(v, 0.0) for v in path[1:-1])
            x = pl.LpVariable(f"x_{rid}_{pid}", lowBound=0, cat='Continuous')
            x_vars[(rid, pid)] = {'var': x, 'path': path}

            # Demand expression
            demand_expr += x

            # Edge capacity expressions
            for e in zip(path[:-1], path[1:]):
                if e not in edgecap_constr:
                    edgecap_constr[e] = pl.LpAffineExpression()
                edgecap_constr[e] += x

            # Node incoming expressions (excluding source and target)
            for v in path[1:]:
                if v != s_r and v != t_r:
                    if v not in node_in_constr:
                        node_in_constr[v] = pl.LpAffineExpression()
                    node_in_constr[v] += x

        # Add demand constraint
        demand_constr[rid] = prob.addConstraint(demand_expr == d_r, name=f"demand_{rid}")

    # Add edge capacity constraints
    for (u, v), expr in edgecap_constr.items():
        edgecap_constr[(u, v)] = prob.addConstraint(expr <= C * y_vars[(u, v)], name=f"edgecap_{u}_{v}")

    # Add node incoming constraints
    for v, expr in node_in_constr.items():
        cap = transfer_max.get(v, float('inf'))
        node_in_constr[v] = prob.addConstraint(expr <= cap, name=f"node_in_{v}")

    # Set objective: sum of edge prices * y_vars + transit costs * x_vars
    obj = pl.LpAffineExpression()
    for (u, v), y in y_vars.items():
        obj += G[u][v]['price'] * y
    for (rid, pid), rec in x_vars.items():
        x = rec['var']
        transit_cost = sum(transfer_price.get(v, 0.0) for v in rec['path'][1:-1])
        obj += transit_cost * x
    prob.setObjective(obj)

    return prob, x_vars, y_vars, demand_constr, edgecap_constr, node_in_constr


In [29]:
master, x_vars, y_vars, demand_constr, edgecap_constr, node_in_constr = build_initial_master(
    G, reqs, paths_for_req, C, transfer_price, transfer_max
)
print("Initial master LP built. Initial columns per request:", {k: len(v) for k, v in paths_for_req.items()})


Initial master LP built. Initial columns per request: {0: 6, 1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6, 8: 6, 9: 6, 10: 6, 11: 6, 12: 6, 13: 6, 14: 6, 15: 6, 16: 6, 17: 6, 18: 6, 19: 6, 20: 6, 21: 6, 22: 6, 23: 6, 24: 6, 25: 6, 26: 6, 27: 6, 28: 6, 29: 6, 30: 6, 31: 6, 32: 6, 33: 6, 34: 6, 35: 6, 36: 6, 37: 6, 38: 6, 39: 6, 40: 6, 41: 6, 42: 6, 43: 6, 44: 6, 45: 6, 46: 6, 47: 6, 48: 6, 49: 6, 50: 6, 51: 6, 52: 6, 53: 6, 54: 6, 55: 6, 56: 6, 57: 6, 58: 6, 59: 6, 60: 6, 61: 6, 62: 6, 63: 6, 64: 6, 65: 6, 66: 6, 67: 6, 68: 6, 69: 6, 70: 6, 71: 6, 72: 6, 73: 6, 74: 6, 75: 6, 76: 6, 77: 6, 78: 6, 79: 6, 80: 6, 81: 6, 82: 6, 83: 6, 84: 6, 85: 6, 86: 6, 87: 6, 88: 6, 89: 6, 90: 6, 91: 6, 92: 6, 93: 6, 94: 6, 95: 6, 96: 6, 97: 6, 98: 6, 99: 6, 100: 6, 101: 6, 102: 6, 103: 6, 104: 6, 105: 6, 106: 6, 107: 6, 108: 6, 109: 6, 110: 6, 111: 6, 112: 6, 113: 6, 114: 6, 115: 6, 116: 6, 117: 6, 118: 6, 119: 6, 120: 6, 121: 6, 122: 6, 123: 6, 124: 6, 125: 6, 126: 6, 127: 6, 128: 6, 129: 6, 130: 6, 131: 6,

In [30]:
def extract_duals(model, demand_constr, edgecap_constr, node_in_constr):
    # Solve the model with duals enabled
    solver = pl.PULP_CBC_CMD(msg=0, timeLimit=TIME_LIMIT_LP, warmStart=True, keepFiles=True)
    model.setSolver(solver)
    model.solve()

    # Check solver status
    status = pl.LpStatus[model.status]
    if status != 'Optimal':
        # If infeasible, try to diagnose
        if status == 'Infeasible':
            print("Model is infeasible. Writing LP file for debugging.")
            model.write('master_lp.lp')
        raise RuntimeError(f"Master LP solve failed or not optimal. Status: {status}")

    # Extract duals
    pi = {}     # demand duals
    beta = {}   # edgecap duals
    gamma = {}  # node_in duals

    for k in demand_constr:
        constr = model.constraints[f"demand_{k}"]
        pi[k] = constr.pi if constr.pi is not None else 0.0
    for e in edgecap_constr:
        constr = model.constraints[f"edgecap_{e[0]}_{e[1]}"]
        beta[e] = constr.pi if constr.pi is not None else 0.0
    for v in node_in_constr:
        constr = model.constraints[f"node_in_{v}"]
        gamma[v] = constr.pi if constr.pi is not None else 0.0

    obj_val = pl.value(model.objective) if model.objective is not None else float('inf')
    return pi, beta, gamma, obj_val

def pricing_for_request(rk, pi, beta, gamma):
    s, t = rk['s'], rk['t']
    H = nx.DiGraph()
    for u, v in G.edges():
        be = beta.get((u, v), 0.0)
        node_cost = 0.0
        if v != t:
            node_cost = transfer_price.get(v, 0.0) + gamma.get(v, 0.0)
        w = be + node_cost
        H.add_edge(u, v, weight=w)
    try:
        p = nx.shortest_path(H, s, t, weight='weight')
    except Exception:
        return None, None
    path_edges = list(zip(p[:-1], p[1:]))
    sum_beta = sum(beta.get(e, 0.0) for e in path_edges)
    sum_gamma = sum(gamma.get(v, 0.0) for v in p[1:-1])
    path_transit = sum(transfer_price.get(v, 0.0) for v in p[1:-1])
    pi_k = pi[rk['id']]
    return p, (path_transit + sum_beta + sum_gamma - pi_k)

In [31]:
for r in reqs:
    if r['id'] not in paths_for_req or len(paths_for_req[r['id']]) == 0:
        print("NO PATH for req", r)
        
missing_edges = set()
for rid, paths in paths_for_req.items():
    for p in paths:
        for e in zip(p[:-1], p[1:]):
            if e not in G.edges():
                missing_edges.add(e)
            if e not in y_vars:
                missing_edges.add(("no_yvar", e))
print("missing edges or y_vars:", missing_edges)

total_demand = sum(r['d'] for r in reqs)
total_in_node_caps = sum(v for v in transfer_max.values() if v is not None)
print("total demand", total_demand, "sum node incoming caps", total_in_node_caps)

print("Edges missing in y_vars:", set(edgecap_constr.keys()) - set(y_vars.keys()))


missing edges or y_vars: set()
total demand 523693.0 sum node incoming caps 42000.0
Edges missing in y_vars: set()


In [32]:
iter_count = 0
while True:
    iter_count += 1
    print(f"\n=== CG iteration {iter_count} ===")

    # Solve master LP and extract duals
    pi, beta, gamma, lp_obj = extract_duals(master, demand_constr, edgecap_constr, node_in_constr)
    print(f"Master LP objective: {lp_obj:.6f}")

    # Pricing: for each request, find best path and its reduced cost
    new_columns = 0
    for r in reqs:
        rid = r['id']
        p, path_cost_sum = pricing_for_request(r, pi, beta, gamma)
        if p is None:
            continue

        redcost = path_cost_sum - pi[rid]
        print(f"req {rid}: best path {p} reduced cost = {redcost:.6e}")

        if redcost < -EPS:
            # Build new x variable
            x = pl.LpVariable(f"x_{rid}_cg{iter_count}", lowBound=0, cat='Continuous')
            x_vars[(rid, f"cg{iter_count}")] = {'var': x, 'path': p}

            # Update demand constraint
            demand_expr = pl.LpAffineExpression()
            for (rid2, pid), rec in x_vars.items():
                if rid2 == rid:
                    demand_expr += rec['var']
            master.constraints[f"demand_{rid}"] = demand_expr == r['d']

            # Update edge capacity constraints
            for e in zip(p[:-1], p[1:]):
                if e not in edgecap_constr:
                    edgecap_constr[e] = pl.LpAffineExpression()
                    edgecap_constr[e] = master.addConstraint(edgecap_constr[e] <= C * y_vars[e], name=f"edgecap_{e[0]}_{e[1]}")
                edgecap_expr = pl.LpAffineExpression()
                for (rid2, pid), rec in x_vars.items():
                    path = rec['path']
                    if e in list(zip(path[:-1], path[1:])):
                        edgecap_expr += rec['var']
                master.constraints[f"edgecap_{e[0]}_{e[1]}"] = edgecap_expr <= C * y_vars[e]

            # Update node incoming constraints
            for v in p[1:]:
                if v != r['s'] and v != r['t']:
                    if v not in node_in_constr:
                        node_in_constr[v] = pl.LpAffineExpression()
                        cap = transfer_max.get(v, float('inf'))
                        node_in_constr[v] = master.addConstraint(node_in_constr[v] <= cap, name=f"node_in_{v}")
                    node_in_expr = pl.LpAffineExpression()
                    for (rid2, pid), rec in x_vars.items():
                        path = rec['path']
                        if v in path[1:-1]:
                            node_in_expr += rec['var']
                    master.constraints[f"node_in_{v}"] = node_in_expr <= transfer_max.get(v, float('inf'))

            # Update objective
            obj = pl.LpAffineExpression()
            for (u, v), y in y_vars.items():
                obj += G[u][v]['price'] * y
            for (rid, pid), rec in x_vars.items():
                x_var = rec['var']
                transit_cost = sum(transfer_price.get(v, 0.0) for v in rec['path'][1:-1])
                obj += transit_cost * x_var
            master.setObjective(obj)

            new_columns += 1
            print(f"  -> Added column for req {rid}, path {p}")

    if new_columns == 0:
        print("No columns with negative reduced cost found. Stopping CG.")
        break
    if iter_count >= MAX_CG_ITERS:
        print("Reached max CG iterations. Stopping.")
        break

# Final LP solve
master.solve()
print("Final master LP objective:", pl.value(master.objective))

# Solve MILP: make y variables integer
print("\nSolving final MILP on generated columns (make y integer)...")
for e, yv in y_vars.items():
    yv.cat = 'Integer'
solver = pl.PULP_CBC_CMD(msg=1, timeLimit=TIME_LIMIT_MIP, warmStart=True)
master.setSolver(solver)
master.solve()

if pl.LpStatus[master.status] in ['Optimal', 'Not Solved']:
    print("MIP finished with status", pl.LpStatus[master.status])
    mip_obj = pl.value(master.objective)
    print("MIP objective:", mip_obj)
    sol_y = {e: int(round(pl.value(y_vars[e]))) for e in y_vars if pl.value(y_vars[e]) is not None and pl.value(y_vars[e]) > 1e-9}
    sol_x = {}
    for (rid, pid), rec in x_vars.items():
        var = rec['var']
        val = pl.value(var) if pl.value(var) is not None else 0.0
        if val > 1e-8:
            sol_x[(rid, pid)] = {'flow': val, 'path': rec['path']}
    print("\nSelected y (edge -> vehicles):")
    for e, cnt in sol_y.items():
        print(f"  {e}: {cnt}")
    print("\nSelected flows x (req,path,flow):")
    for (rid, pid), rec in sol_x.items():
        print(f"  req {rid} path {rec['path']} flow {rec['flow']:.3f}")
else:
    raise RuntimeError(f"MIP didn't return feasible/optimal solution; status: {pl.LpStatus[master.status]}")



=== CG iteration 1 ===
Master LP objective: 685588670700.428345
req 116: best path [np.int64(19880), np.float64(12355.0)] reduced cost = -4.038151e+07
  -> Added column for req 116, path [np.int64(19880), np.float64(12355.0)]
req 1334: best path [np.int64(12352), np.float64(18441.0), np.int64(10326)] reduced cost = -4.036889e+07
  -> Added column for req 1334, path [np.int64(12352), np.float64(18441.0), np.int64(10326)]
req 3418: best path [np.int64(12868), np.float64(18441.0), np.int64(10326)] reduced cost = -3.998853e+07
  -> Added column for req 3418, path [np.int64(12868), np.float64(18441.0), np.int64(10326)]
req 3630: best path [np.int64(18219), np.float64(18441.0), np.int64(10326)] reduced cost = -3.955567e+07
  -> Added column for req 3630, path [np.int64(18219), np.float64(18441.0), np.int64(10326)]

=== CG iteration 2 ===
Master LP objective: 685588670700.428345
req 3418: best path [np.int64(12868), np.float64(11209.0), np.int64(10326)] reduced cost = -3.261597e+07
  -> Adde